# Módulo 4: Teoría Moderna de Carteras y Optimización
Incluye frontera eficiente, restricciones prácticas y Black-Litterman.

## 1. Optimización cuadrática de Markowitz

El modelo de **Markowitz** (Modern Portfolio Theory, 1952) busca la combinación de activos que **minimiza la varianza de la cartera** (riesgo), dado un nivel de rendimiento esperado, o equivalentemente, maximiza el rendimiento para un nivel de riesgo.

### Supuestos clave
- Los retornos esperados y las covarianzas entre activos son conocidas.
- Los inversores son aversos al riesgo.
- El riesgo se mide con la varianza de la cartera.

### Formulación matemática

Sea:
- $ \mu $: vector de rendimientos esperados (n activos).
- $ \Sigma $: matriz de covarianzas (n x n).
- $ w $: vector de pesos de la cartera.
- $ \mu_p = w^\top \mu $: rendimiento esperado de la cartera.
- $ \sigma_p^2 = w^\top \Sigma w $: varianza de la cartera.

El problema estándar es:

$$
\min_w \; w^\top \Sigma w
$$

sujeto a:
- $ w^\top \mu = \mu_p $ (rendimiento objetivo),
- $ \mathbf{1}^\top w = 1 $ (los pesos suman 1),
- opcionalmente $ w \geq 0 $ (sin posiciones cortas).

Este es un problema de **programación cuadrática (QP)** y se resuelve fácilmente con librerías como `cvxpy` en Python.

El resultado es la **frontera eficiente**, una curva convexa que muestra las carteras óptimas en el plano (riesgo, rendimiento).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
returns = np.random.randn(1000, 4) * 0.01
cov = np.cov(returns.T)
mean_returns = returns.mean(axis=0)

weights = np.random.dirichlet(np.ones(4), 5000)
port_returns = weights.dot(mean_returns)
port_vols = np.sqrt(np.sum(weights @ cov * weights, axis=1))

plt.scatter(port_vols, port_returns, alpha=0.3)
plt.xlabel('Volatilidad')
plt.ylabel('Retorno esperado')
plt.title('Frontera eficiente simulada')
plt.show()

### Ejercicio:
1. Implementa la optimización cuadrática de Markowitz.
2. Introduce Black-Litterman con opiniones de inversores.